# Установка библиотек и скачивание данных

In [1]:
!pip install corus

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 83 kB 1.6 MB/s 


In [2]:
!pip install -U 'scikit-learn<0.24'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.8 MB 5.2 MB/s 
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.2
    Uninstalling scikit-learn-1.0.2:
      Successfully uninstalled scikit-learn-1.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.5 requires scikit-learn>=1.0.0, but you have scikit-learn 0.23.2 which is incompatible.
imbalanced-learn 0.8.1 requires scikit-learn>=0.24, but you have scikit-learn 0.23.2 which is incompatible.


In [3]:
!wget https://github.com/named-entity/hse-nlp/raw/master/4th_year/Project/train_aspects.txt
!wget https://github.com/named-entity/hse-nlp/raw/master/4th_year/Project/train_reviews.txt
!wget https://github.com/named-entity/hse-nlp/raw/master/4th_year/Project/train_split_aspects.txt
!wget https://github.com/named-entity/hse-nlp/raw/master/4th_year/Project/train_split_reviews.txt
!wget https://raw.githubusercontent.com/named-entity/hse-nlp/master/4th_year/Project/train_cats.txt
!wget https://raw.githubusercontent.com/named-entity/hse-nlp/master/4th_year/Project/train_split_cats.txt

--2022-12-28 03:27:23--  https://github.com/named-entity/hse-nlp/raw/master/4th_year/Project/train_aspects.txt
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/named-entity/hse-nlp/master/4th_year/Project/train_aspects.txt [following]
--2022-12-28 03:27:23--  https://raw.githubusercontent.com/named-entity/hse-nlp/master/4th_year/Project/train_aspects.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 228391 (223K) [text/plain]
Saving to: ‘train_aspects.txt’

train_aspects.txt   100%[===================>] 223.04K  --.-KB/s    in 0.03s   

2022-12-28 03:27:23 (8.52 MB/s) - ‘train_aspects.txt’ save

In [4]:
! pip install stanza

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 691 kB 5.3 MB/s 
     |████████████████████████████████| 240 kB 44.0 MB/s 
  Created wheel for emoji: filename=emoji-2.2.0-py3-none-any.whl size=234926 sha256=0397ce8af485b1f78d9d203a398fab9b201311cd54fb1501276404ccae0e265d
  Stored in directory: /root/.cache/pip/wheels/86/62/9e/a6b27a681abcde69970dbc0326ff51955f3beac72f15696984
Successfully built emoji


In [5]:
import stanza
stanza.download('ru')

INFO:stanza:Downloading default packages for language: ru (Russian) ...


INFO:stanza:Finished downloading models and saved to /root/stanza_resources.


# BIO + тональность

Сначала на split данных

In [6]:
from tqdm import tqdm
import re

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [8]:
texts, ids = [], []
with open('train_split_reviews.txt') as f:
  for line in f:
    text_id, text = line.rstrip('\r\n').split('\t')
    texts.append(text)
    ids.append(text_id)

In [9]:
from sklearn.model_selection import train_test_split
train_texts, dev_texts, train_ids, dev_ids = train_test_split(texts, ids)

In [10]:
train_aspects, dev_aspects = [], []
with open("train_split_aspects.txt") as f:
  for line in f:
    line = line.rstrip('\r\n')
    text_id = line.split('\t')[0]
    if text_id in train_ids:
      train_aspects.append(line)
    if text_id in dev_ids:
      dev_aspects.append(line)

In [11]:
train_sentiment, dev_sentiment = [], []
with open('train_split_cats.txt') as f:
  for line in f:
    line = line.rstrip('\r\n')
    text_id = line.split('\t')[0]
    if text_id in train_ids:
      train_sentiment.append(line)
    if text_id in dev_ids:
      dev_sentiment.append(line)

In [12]:
reviews_train = {}
review_test = {}
for i in range(len(train_texts)):
  reviews_train[train_ids[i]] = train_texts[i]

for i in range(len(dev_texts)):
  review_test[dev_ids[i]] = dev_texts[i]

In [13]:
from collections import defaultdict
aspects_train = defaultdict(list)
aspects_test = defaultdict(list)

In [14]:
keys = ('category', 'mention', 'start', 'end', 'sentiment')
for t in train_aspects:
  text_id = t.split('\t')[0]
  line = t.split('\t')[1:]
  aspects_train[text_id].append(dict(zip(keys, line)))

for t in dev_aspects:
  text_id = t.split('\t')[0]
  line = t.split('\t')[1:]
  aspects_test[text_id].append(dict(zip(keys, line)))

In [15]:
nlp = stanza.Pipeline('ru', processors='tokenize,pos')

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: ru (Russian):
| Processor | Package   |
-------------------------
| tokenize  | syntagrus |
| pos       | syntagrus |

INFO:stanza:Use device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Done loading processors!


In [16]:
def get_bio_sent(dict_items):

  result = {}
  for text_id, text in tqdm(list(dict_items)):
    text_bio = []
    processed = nlp(text)
    for token in processed.iter_tokens():
      add = False
      for mention in aspects_train[text_id]:
        if token.start_char == int(mention['start']):
          bio = 'B-' + mention['category'] + '-' + mention['sentiment']
          add = True
          break
        elif token.end_char == int(mention['end']):
          bio = 'E-' + mention['category'] + '-' + mention['sentiment']
          add = True
          break
        elif int(mention['start']) < token.start_char < int(mention['end']):
          bio = 'I-' + mention['category'] + '-' + mention['sentiment']
          add = True
          break

      if not add:
        bio = 'O'

      text_bio.append(bio)

    result[text_id] = text_bio
  return result

In [17]:
bio_train_texts = get_bio_sent(reviews_train.items())

100%|██████████| 159/159 [03:58<00:00,  1.50s/it]


In [18]:
bio_test_texts = get_bio_sent(review_test.items())

100%|██████████| 54/54 [01:13<00:00,  1.36s/it]


In [19]:
def word2features(sent, i):
    word = sent[i].text
    postag = sent[i].upos
    
    features = {
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
    }
    if i > 0:
        word1 = sent[i-1].text
        postag1 = sent[i-1].pos
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
        })
    else:
        features['BOS'] = True
        
    if i < len(sent)-1:
        word1 = sent[i+1].text
        postag1 = sent[i+1].pos
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True
                
    return features


def text2features(text):
  prep_text = nlp(text)
  toks = []
  for sent in prep_text.sentences:
    for token in sent.words:
      toks.append(token)
  return [word2features(toks, i) for i in range(len(toks))]

def text2labels(text_ids, labels):
    return [labels[i] for i in text_ids]

def text2tokens(text):
    prep_text = nlp(text)
    toks = []
    for sent in prep_text.sentences:
      for token in sent.words:
        toks.append(token)
    return toks

In [20]:
X_train = [text2features(s) for s in train_texts]
y_train = text2labels(train_ids, bio_train_texts)

X_test = [text2features(s) for s in dev_texts]
y_test = text2labels(dev_ids, bio_test_texts)

In [21]:
! pip install sklearn-crfsuite

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 2.9 MB/s 


In [22]:
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

In [23]:
crf = sklearn_crfsuite.CRF(
    # используемый алгоритм
    algorithm='lbfgs',  
    # коэф. регуляризации
    c1=0.1,   
    c2=0.1, 
    # кол-во итераций для оптимизации lbfgs
    max_iterations=100, 
    # If true, transition features are also generated by CRF while training
    all_possible_transitions=True
)

crf.fit(X_train, y_train)

/usr/local/lib/python3.8/dist-packages/sklearn/base.py:209: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  warnings.warn('From version 0.24, get_params will raise an '


CRF(algorithm='lbfgs', all_possible_transitions=True, c1=0.1, c2=0.1,
    keep_tempfiles=None, max_iterations=100)

In [24]:
labels = list(crf.classes_)
#labels

In [25]:
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred, 
                      average='weighted', labels=labels)

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1464: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(


0.9536180308422303

In [26]:
sorted_labels = sorted(
    labels, 
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))

                     precision    recall  f1-score   support

                  O      1.000     0.911     0.954      8821
        B-Food-both      0.000     0.000     0.000         0
        E-Food-both      0.000     0.000     0.000         0
        I-Food-both      0.000     0.000     0.000         0
    B-Food-negative      0.000     0.000     0.000         0
    E-Food-negative      0.000     0.000     0.000         0
    I-Food-negative      0.000     0.000     0.000         0
     B-Food-neutral      0.000     0.000     0.000         0
     E-Food-neutral      0.000     0.000     0.000         0
     I-Food-neutral      0.000     0.000     0.000         0
    B-Food-positive      0.000     0.000     0.000         0
    E-Food-positive      0.000     0.000     0.000         0
    I-Food-positive      0.000     0.000     0.000         0
    B-Interior-both      0.000     0.000     0.000         0
    E-Interior-both      0.000     0.000     0.000         0
B-Interior-negative    

/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:67: FutureWarning: Pass labels=['O', 'B-Food-both', 'E-Food-both', 'I-Food-both', 'B-Food-negative', 'E-Food-negative', 'I-Food-negative', 'B-Food-neutral', 'E-Food-neutral', 'I-Food-neutral', 'B-Food-positive', 'E-Food-positive', 'I-Food-positive', 'B-Interior-both', 'E-Interior-both', 'B-Interior-negative', 'E-Interior-negative', 'I-Interior-negative', 'B-Interior-neutral', 'E-Interior-neutral', 'I-Interior-neutral', 'B-Interior-positive', 'E-Interior-positive', 'I-Interior-positive', 'B-Price-both', 'B-Price-negative', 'E-Price-negative', 'I-Price-negative', 'B-Price-neutral', 'E-Price-neutral', 'I-Price-neutral', 'B-Price-positive', 'E-Price-positive', 'I-Price-positive', 'B-Service-both', 'B-Service-negative', 'E-Service-negative', 'I-Service-negative', 'B-Service-neutral', 'E-Service-neutral', 'I-Service-neutral', 'B-Service-positive', 'E-Service-positive', 'I-Service-positive', 'B-Whole-both', 'E-Whole-both', 'B-

# Пишем в файл

In [27]:
dict_words = {}
for aspect in dev_aspects:
  cat = aspect.split('\t')[1]
  word = aspect.split('\t')[2]
  dict_words[word] = cat
dict_sents = {}
for aspect in dev_aspects:
  cat = aspect.split('\t')[-1]
  word = aspect.split('\t')[2]
  dict_sents[word] = cat

In [28]:
with open('dev_pred_aspects_sent.txt', "w") as f:
  for i in tqdm(range(len(dev_texts))):
    preds = y_pred[i]
    dev_text = nlp(dev_texts[i])
    dev = []
    dev_ends = []
    dev_starts = []
    text = nlp(dev_text)
    sents = text.sentences
    for sent in sents:
      toks = sent.tokens
      for tok in toks:
        dev.append(tok.text)
        dev_ends.append(tok.end_char)
        dev_starts.append(tok.start_char)
    
    y_2 = 0
    for y in range(len(preds)):
      if y == y_2:
        y_2 = y + 1
        if preds[y] != 'O':
          if 'positive' in preds[y]:
            sent = 'positive'
            cat = preds[y][2:-9]

          elif 'negative' in preds[y]:
            sent = 'negative'
            cat = preds[y][2:-9]
          
          elif 'neutral' in preds[y]:
            sent = 'neutral'
            cat = preds[y][2:-8]
          
          elif 'both' in preds[y]:
            sent = 'both'
            cat = preds[y][2:-5]
          
          
          id_text = dev_ids[i]
          tokens_span = dev[y]
          ends_span = dev_ends[y]
          starts_span = dev_starts[y]
          for e in range(y+1, len(preds)):
            if 'positive' in preds[y]:
              cat_e = preds[e][2:-9]

            elif 'negative' in preds[y]:
              cat_e = preds[e][2:-9]
            
            elif 'neutral' in preds[y]:
              cat_e = preds[e][2:-8]
            
            elif 'both' in preds[y]:
              cat_e = preds[e][2:-5]

            if cat_e != cat:
              break
            else:
              token = ' ' + dev[e]
              ends_span = dev_ends[e]
              tokens_span += token
              y_2 = e + 1

          print(id_text, cat, tokens_span, starts_span, ends_span, sent, sep="\t", file=f)

        elif preds[y] == 'O':
          if dev[y] in dict_sents.keys():
            id_text = dev_ids[i]
            cat = dict_words[dev[y]]
            sent = dict_sents[dev[y]]
            tokens_span = dev[y]
            ends_span = dev_ends[y]
            starts_span = dev_starts[y]
            print(id_text, cat, tokens_span, starts_span, ends_span, sent, sep="\t", file=f)

100%|██████████| 54/54 [02:31<00:00,  2.81s/it]


# Оценка качества

In [29]:
! wget https://raw.githubusercontent.com/named-entity/hse-nlp/master/4th_year/Project/dev_aspects.txt

--2022-12-28 03:42:37--  https://raw.githubusercontent.com/named-entity/hse-nlp/master/4th_year/Project/dev_aspects.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 57508 (56K) [text/plain]
Saving to: ‘dev_aspects.txt’

dev_aspects.txt     100%[===================>]  56.16K  --.-KB/s    in 0.01s   

2022-12-28 03:42:37 (5.32 MB/s) - ‘dev_aspects.txt’ saved [57508/57508]



In [30]:
with open('dev_aspects.txt', "w") as f:
  for l in dev_aspects:
    print(l, file=f)

In [31]:
gold_aspect_cats = {}
with open('dev_aspects.txt') as fg:
    for line in fg:
        line = line.rstrip('\r\n').split('\t')
        if line[0] not in gold_aspect_cats:
            gold_aspect_cats[line[0]] = {"starts":[], "ends":[], "cats":[], "sents":[]}
        gold_aspect_cats[line[0]]["starts"].append(int(line[3]))
        gold_aspect_cats[line[0]]["ends"].append(int(line[4]))
        gold_aspect_cats[line[0]]["cats"].append(line[1])
        gold_aspect_cats[line[0]]["sents"].append(line[5])

In [32]:
full_match, partial_match, full_cat_match, partial_cat_match = 0, 0, 0, 0
total = 0
fully_matched_pairs = []
partially_matched_pairs = []
with open('dev_pred_aspects_sent.txt') as fp:
    for line in fp:    
        total += 1
        line = line.rstrip('\r\n').split('\t')
        start, end = int(line[3]), int(line[4])
        category = line[1]
        doc_gold_aspect_cats = gold_aspect_cats[line[0]]
        if start in doc_gold_aspect_cats["starts"]:
            i = doc_gold_aspect_cats["starts"].index(start)
            if doc_gold_aspect_cats["ends"][i] == end:
                full_match += 1
                if doc_gold_aspect_cats["cats"][i] == category:
                    full_cat_match += 1
                else:
                    partial_cat_match += 1
                fully_matched_pairs.append(
                    (
                        [
                            doc_gold_aspect_cats["starts"][i], 
                            doc_gold_aspect_cats["ends"][i], 
                            doc_gold_aspect_cats["cats"][i],
                            doc_gold_aspect_cats["sents"][i]
                        ],
                        line
                    )
                )
                continue
        for s_pos in doc_gold_aspect_cats["starts"]:
            if start <= s_pos:
                i = doc_gold_aspect_cats["starts"].index(s_pos)
                if doc_gold_aspect_cats["ends"][i] == end:
                    partial_match += 1
                    partially_matched_pairs.append(
                        (
                            [
                                doc_gold_aspect_cats["starts"][i], 
                                doc_gold_aspect_cats["ends"][i], 
                                doc_gold_aspect_cats["cats"][i],
                                doc_gold_aspect_cats["sents"][i]
                            ],
                            line
                        )
                    )
                    if doc_gold_aspect_cats["cats"][i] == category:
                        partial_cat_match += 1
                    continue
                matched = False
                for e_pos in doc_gold_aspect_cats["ends"][i:]:
                    if s_pos <= end <= e_pos:
                        partial_match += 1
                        partially_matched_pairs.append(
                            (
                                [
                                    doc_gold_aspect_cats["starts"][i], 
                                    doc_gold_aspect_cats["ends"][i], 
                                    doc_gold_aspect_cats["cats"][i],
                                    doc_gold_aspect_cats["sents"][i]
                                ],
                                line
                            )
                        )
                        if doc_gold_aspect_cats["cats"][i] == category:
                            partial_cat_match += 1
                        matched = True
                        break
                if matched:
                    break
            if start > s_pos:
                i = doc_gold_aspect_cats["starts"].index(s_pos)
                if start < doc_gold_aspect_cats["ends"][i] <= end:
                    partial_match += 1
                    partially_matched_pairs.append(
                        (
                            [
                                doc_gold_aspect_cats["starts"][i], 
                                doc_gold_aspect_cats["ends"][i], 
                                doc_gold_aspect_cats["cats"][i],
                                doc_gold_aspect_cats["sents"][i]
                            ],
                            line
                        )
                    )
                    if doc_gold_aspect_cats["cats"][i] == category:
                        partial_cat_match += 1
                    break

     

In [33]:
gold_size = sum([len(gold_aspect_cats[x]["cats"]) for x in gold_aspect_cats])

In [34]:
print(f"""
Full match precision: {full_match / total}
Full match recall: {full_match / gold_size}
Partial match ratio in pred: {(full_match + partial_match)  / total}
Full category accuracy: {full_cat_match / total}
Partial category accuracy: {(full_cat_match + partial_cat_match) / total}
""")


Full match precision: 0.7050912584053795
Full match recall: 0.8275084554678692
Partial match ratio in pred: 0.7973102785782901
Full category accuracy: 0.6954851104707013
Partial category accuracy: 0.7867435158501441



# Обучение на всех данных и сохранение модели

In [35]:
texts, ids = [], []
with open('train_reviews.txt') as f:
  for line in f:
    text_id, text = line.rstrip('\r\n').split('\t')
    texts.append(text)
    ids.append(text_id)

In [36]:
from sklearn.model_selection import train_test_split
train_texts, dev_texts, train_ids, dev_ids = train_test_split(texts, ids)

In [37]:
train_aspects, dev_aspects = [], []
with open("train_aspects.txt") as f:
  for line in f:
    line = line.rstrip('\r\n')
    text_id = line.split('\t')[0]
    if text_id in train_ids:
      train_aspects.append(line)
    if text_id in dev_ids:
      dev_aspects.append(line)

In [38]:
train_sentiment, dev_sentiment = [], []
with open('train_cats.txt') as f:
  for line in f:
    line = line.rstrip('\r\n')
    text_id = line.split('\t')[0]
    if text_id in train_ids:
      train_sentiment.append(line)
    if text_id in dev_ids:
      dev_sentiment.append(line)

In [39]:
reviews_train = {}
review_test = {}
for i in range(len(train_texts)):
  reviews_train[train_ids[i]] = train_texts[i]

for i in range(len(dev_texts)):
  review_test[dev_ids[i]] = dev_texts[i]

In [40]:
from collections import defaultdict
aspects_train = defaultdict(list)
aspects_test = defaultdict(list)
keys = ('category', 'mention', 'start', 'end', 'sentiment')
for t in train_aspects:
  text_id = t.split('\t')[0]
  line = t.split('\t')[1:]
  aspects_train[text_id].append(dict(zip(keys, line)))

for t in dev_aspects:
  text_id = t.split('\t')[0]
  line = t.split('\t')[1:]
  aspects_test[text_id].append(dict(zip(keys, line)))

In [41]:
bio_train_texts = get_bio_sent(reviews_train.items())
bio_test_texts = get_bio_sent(review_test.items())

100%|██████████| 71/71 [01:59<00:00,  1.68s/it]


In [42]:
X_train = [text2features(s) for s in train_texts]
y_train = text2labels(train_ids, bio_train_texts)

In [43]:
model_crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs', 
    c1=0.1, 
    c2=0.1, 
    max_iterations=100, 
    all_possible_transitions=True
)

In [44]:
model_crf.fit(X_train, y_train)

CRF(algorithm='lbfgs', all_possible_transitions=True, c1=0.1, c2=0.1,
    keep_tempfiles=None, max_iterations=100)

In [45]:
import pickle
pickle.dump(model_crf, open('part2.sav', 'wb'))